### __WordsOfChessAndAI - Metadati__ 

#### __Obiettivo__

L'obiettivo del Jupyter Notebook presentato consiste nell'estrazione di alcune informazioni mediante l'impiego di librerie in grado di leggere e modificare il contenuto di file PDF. Pertanto, di seguito sono definite le funzioni necessarie per estrapolare dal contesto tali dati, suddivisi in:
- DOI
- Titolo
- Autore
- Abstract

_DOI_ è l'acronimo di __Digital Object Identifier__, identificativo univoco di risorse digitali. Per poter apprendere tale informazione è necessario utilizzare la _REST API_ fornita da __CrossRef__; CrossRef è un'infrastuttura digitale dedita alle memorizzazione di tutti gli articoli posti in ambito accademico. Di seguito, si percepisce l'importanza del titolo e dell'autore per ogni PDF, affinchè interrogando la _Application Programming Interface_ sia possibile ottenere il DOI.

In [108]:
target_word = "abstract"
main_path = "../articles/"
crossref_url = "https://api.crossref.org/works/10.1037/0003-066X.59.1.29/agency"

In [109]:
import os

def get_path_files(path: str) -> list:
    size = 0
    path_files = []

    for item in os.listdir(path):
        if not item.endswith(".pdf"):
            print("Not a pdf", item, "\n")
            continue
        else:
            path_files.append(path + item)
            size += 1

    return path_files

path_files = get_path_files(main_path)

Ottenuto ogni singolo percorso dei file, provvedo a convertire il formato PDF nelle immagini appartenenti, in maniera tale che attraverso l'__OCR PyTesseract__ sia in grado di estrarre il testo corrispondente. È possibile notare anche la presenza di una variabile denominata _index_; definisce la linea in cui sia stata individuata la key-word
"__Abstract__", affinchè, successivamente a qualche attività di manipolazione di stringhe, sia capace di estrapolare il contenuto della sezione.

In [110]:
import pytesseract

from typing import Dict
from pdf2image import convert_from_path

class ScannedText:
    def __init__(self, index: int, text: str):
        self.index = index
        self.text = text

def convert_file_to_images(path: str) -> list:
    try:
        return convert_from_path(path)[:2]
    except Exception as e:
        print("Error during conversion from file to image:", e, "\n")

def get_target_word_index(text: str) -> int:
    count_line = 0

    lines = text.splitlines()
    for line in lines:
        if "abstract" in line:
            return count_line
        
        count_line += 1
    
    return -1

def scan_file_to_text(paths: list[str]) -> dict[str, ScannedText]:
    dict_scanned_texts: Dict[str, ScannedText] = {}

    for path in paths:
        images = convert_file_to_images(path)

        text = ""
        try:
            for image in images:
                text = text + pytesseract.image_to_string(image)

            dict_scanned_texts[path] = ScannedText(get_target_word_index(text.lower()), text)
        except Exception as e:
            print("Error during conversion from image to text:", e) 
        
    return dict_scanned_texts

dict_scanned_texts = scan_file_to_text(path_files)

Riepilogando, il dizionario __dict_scanned_text__ possiede una suddivisione key-value come segue:
- La chiave corrisponde al percorso dello specifico file
- Il valore è un oggetto __ScannedText__, a sua volta composto da un indice attuato per definire la linea in cui sia presente la key-word "_Abstract_" e una stringa corrispondente al testo estratto precedentemente.

In [111]:
from typing import List

def remove_white_spaces(index: int, text: str) -> List[str]:
    lines = text.splitlines()
    first_part = lines[:index + 1]

    start_index = index + 1
    count_line = start_index

    for line in lines[start_index:]:
        if len(line) == 0:
            count_line += 1
            continue
        else:
            break
        
    second_part = lines[count_line:]
    
    return first_part + second_part

def extract_abstract_lines(index: int, lines: list[str]) -> str:
    abstract = ""

    for line in lines[index:]:
        if len(line) == 0:
            break

        abstract += line + "\n"

    return abstract

def extract_abstract(dict: dict[str, ScannedText]) -> dict[str, str]:
    dict_abstracts: Dict[str, str] = {}

    for key in dict.keys():
        value = dict.get(key)

        if value.index > -1:
            abstract_lines = remove_white_spaces(value.index, value.text)
            text = extract_abstract_lines(value.index, abstract_lines)
        else:
            text = "Not found"

        dict_abstracts[key] = text

    return dict_abstracts

dict_abstracts = extract_abstract(dict_scanned_texts)

Il passo successivo prevede l'utilizzo di alcune informazioni per recuperare i _metadati_ mancanti. Mediante l'impiego di alcune librerie, è possibile estrapolare dal PDF dato alcune informazioni al riguardo. Pertanto, tramite la _funzione ricorsiva_ riportata, sono estrapolati tutti i dati necessari per la costruzione dei _metadati_.

In [ ]:
# Potrei ampliare il numero di librerie per l'estrapolazione dei metadati

import pymupdf
import pdfplumber

def get_title_from_dicts(pymupdf: dict[str, str], pdfplumber: dict[str, str]) -> str:
    return (pymupdf.get("title") or pdfplumber.get("title") or "Not found")

def get_author_from_dicts(pymupdf: dict[str, str], pdfplumber: dict[str, str]) -> str:
    return (pymupdf.get("author") or pdfplumber.get("author") or "Not found")

def extract_title_and_author(i: int, len: int, paths: list[str], dict_titles = {}, dict_authors = {}) -> tuple[dict[str, str], dict[str, str]]:
    if len == 0:
        return (dict_titles, dict_authors)
    else:
        metadata_pymupdf = pymupdf.open(paths[i]).metadata
        metadata_pdfplumber = pdfplumber.open(paths[i]).metadata

        title = get_title_from_dicts(metadata_pymupdf, metadata_pdfplumber)
        author = get_author_from_dicts(metadata_pymupdf, metadata_pdfplumber)
        
        dict_titles[paths[i]] = title
        dict_authors[paths[i]] = author

        extract_title_and_author(i + 1, len - 1, paths)
        return (dict_titles, dict_authors)

tuple_dict_titles_authors = extract_title_and_author(0, len(path_files), path_files)

In [ ]:
import time
import requests

from crossref.restful import Works

dict_titles = tuple_dict_titles_authors[0]
dict_authors = tuple_dict_titles_authors[1]

work = Works()
def make_api_call(title: str, author: str) -> str:
    url_request = work.query(bibliographic=title, author=author).url

    # Attardare di un certo delay come definito dalla libreria per ogni richiesta successiva
    time.sleep(2)

    try:
        request = requests.get(url_request)

        if request.status_code == 200:
            response = request.json()

            message = response.get("message")
            items = message.get("items")

            for item in items:
                if any(title in value for value in item.get("title")):
                    return item["DOI"]
                                                
                continue
        else:
            raise Exception("Error during request to CrossRef REST API: bad status code")
    except Exception as e:
        print("Error during request to CrossRef REST API:", e)

dict_DOI: Dict[str, str] = {}
for path in path_files:
    if "Not found" in (dict_titles.get(path), dict_authors.get(path)):
        continue
    else:
        DOI = make_api_call(dict_titles.get(path), dict_authors.get(path))
        dict_DOI[path] = DOI

Error during request to CrossRef REST API: 'NoneType' object is not iterable


Concludendo, ottenuti i dati presenti, sono organizzati attraverso la helper class __Metadata__. Come da definizione dell'_obiettivo_, ogni istanza possiede le informazioni richieste, oltre a presentare anche il percorso del file in questione. Infine è implementato un _dump_ delle informazioni recuperate tramite l'ausilio di un file __json__.

In [114]:
import json

from typing import Dict

class Metadata:
    def __init__(self, DOI, path, title, author, abstract):
        self.DOI = DOI
        self.path = path
        self.title = title
        self.author = author
        self.abstract = abstract

    def get_dict(self) -> Dict[str, Dict[str, str]]:
        return {
            path: {
                "DOI": self.DOI,
                "Title": self.title,
                "Author": self.author,
                "Abstract": self.abstract
            }
        }

list_metadata = []
for path in path_files:
    list_metadata.append(Metadata(dict_DOI.get(path), path, dict_titles.get(path), dict_authors.get(path), dict_abstracts[path]))
 
list_json = []
for metadata in list_metadata:
    list_json.append(metadata.get_dict())

with open("../json/metadata.json", "a") as file:
    json.dump(list_json, file, indent=2)